##  CryptotaxIO Parser

### History:

CryptotaxIO Convert Tables 01 - We are getting started.

CryptotaxIO Convert Tables 02 - Basic conversion of Bitfinex files done.

CryptotaxIO Convert Tables 03 - Made distinction between buy and sell trades in Bitfinex files.

CryptotaxIO Convert Tables 04 - Conversion of Coinbase report of ETH works.

CryptotaxIO Convert Tables 05 - Conversion of all Coinbase reports.

CryptotaxIO Convert Tables 06 - Cleaned up and commented this file. Created test .csv files with artifical data, converted the tables, imported to Cryptotax.io, and generated tax report.

CryptotaxIO Parser 07 - Changed name to CryptotaxIO Parser. Further cleansing and commenting.

### HOW TO 

#### 1. Generate the reports by doing the following:

On Bitfinex go to 'Report', click on 'Deposits', 'Withdrawals', and 'Trades' to extract the reports for the time frame of interest.

On Coinbase go to 'Tools', click on '+ New Report', Select type 'Transaction history', the time frame, and extract all reports for all the wallet of interest.

#### 2. Put the reports created by Bitfinex and Coinbase into the directories 'Bitfinex' and 'Coinbase', respectively.

#### 3. Adjust your account name and the file names in the string variables below.

#### 4. Run this script.

In [1]:
###########################################
## ADJUST YOUR ACCOUNT NAME!
## This script uses this account name for both Coinbase and Bitfinex.
###########################################
str_account_name_Coinbase = 'your_account_name@gmail.com'
str_account_name_Bitfinex = 'your_account_name@gmail.com'

###########################################
## Files names of reports generated by Bitfinex
###########################################
## ADJUST THESE NAMES AND/OR LOCATIONS!
###########################################

str_file_name_Bitfinex_deposit = 'Bitfinex/Bitfinex_2017_Deposits.csv'
str_file_name_Bitfinex_withdrawal = 'Bitfinex/Bitfinex_2017_Withdrawal.csv'
str_file_name_Bitfinex_trades = 'Bitfinex/Bitfinex_2017_Trades.csv'

###########################################
## Files names of reports generated by Coinbase
###########################################
## ADJUST THESE NAMES AND/OR LOCATIONS!
###########################################
str_file_names_Coinbase = ['Coinbase/Coinbase-BTC.csv', 'Coinbase/Coinbase-ETH.csv', 'Coinbase/Coinbase-LTC.csv']

### Import packages

In [2]:
###########################################
# import some python packages
###########################################
import numpy as np
import os
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
%load_ext autoreload
%autoreload 2

###########################################
# define header of table to be imported to CryptoTax.io
###########################################
columns = ['exchange_name', 'account_name', 'trade_date', 'buy_asset', 'sell_asset', 'buy_amount', 'sell_amount', 
             'exchange_order_id', 'fee', 'fee_asset', 'transaction_type']

### Define function for converting trading data

In [3]:
###########################################
## Function for converting trading data from US notation to German format
###########################################
## That is we convert from 2017-12-07 10:26:15 to 07.12.2017 10:26:15.

def convert_trade_date(str_trade_date):

    idx_space = str_trade_date.find(' ')
    idx_hyphen = [pos for pos, char in enumerate(str_trade_date) if char == '-']
    # find DOESN't find several instances of a substring
    # ide_hyphen = str_trade_date.find('-')
    # print(idx_space)
    # print(ide_hyphen)

    str_year = str_trade_date[: idx_hyphen[0]]
    str_month = str_trade_date[idx_hyphen[0] + 1 : idx_hyphen[1]]
    str_day = str_trade_date[idx_hyphen[1] + 1 : idx_space]

    str_trade_date_German = str_day + '.' + str_month + '.' + str_year + ' ' + str_trade_date[idx_space + 1 :]

    str_trade_date_German = str_trade_date_German.replace('"', '')

    # print(str_trade_date_German)
    return str_trade_date_German

### Import files generated by exchange Bitfinex

In [4]:
###########################################
### Read file with deposits
###########################################
bool_file_access = os.access(str_file_name_Bitfinex_deposit, os.R_OK)
if bool_file_access == True:
    print('Now reading .csv file for deposits generated by Bitfinex.\n')
    df_deposits = pd.read_csv(str_file_name_Bitfinex_deposit)
    print(df_deposits.iloc[0, :])
else:
    df_deposits = []
    print('ERROR: File for deposits generated by Bitfinex does not exist or cannot be read!')

###########################################
### Read file with withdrawals
###########################################
bool_file_access = os.access(str_file_name_Bitfinex_withdrawal, os.R_OK)
if bool_file_access == True:
    print('\nNow reading .csv file for withdrawals generated by Bitfinex.\n')
    df_withdrawals = pd.read_csv(str_file_name_Bitfinex_withdrawal)
    print(df_withdrawals)
else:
    df_withdrawals = []
    print('ERROR: File for withdrawals generated by Bitfinex does not exist or cannot be read!')

###########################################
### Read file with trades
###########################################
bool_file_access = os.access(str_file_name_Bitfinex_trades, os.R_OK)
if bool_file_access == True:
    print('\nNow reading .csv file for trades generated by Bitfinex.\n')
    df_trades = pd.read_csv(str_file_name_Bitfinex_trades)
    print(df_trades)
else:
    df_trades = []
    print('ERROR: File for trades generated by Bitfinex does not exist or cannot be read!')

ERROR: File for deposits generated by Bitfinex does not exist or cannot be read!
ERROR: File for withdrawals generated by Bitfinex does not exist or cannot be read!
ERROR: File for trades generated by Bitfinex does not exist or cannot be read!


### Create Pandas frame for converting data

This is how the format for custom import looks like for deposits, withdrawals, and trades:

The following data is taken from the sample .xlsx and .csv files provided by Cryptotax.io:

exchange_name	account_name	trade_date	buy_asset	sell_asset	buy_amount	sell_amount	exchange_order_id	fee	fee_asset	transaction_type

Kraken	Kraken account 1	18.10.2017 20:34:48	ETH		14.99923		R2CM4ND-WGYEY6-KN3XHV	0	ETH	deposit

Kraken	Kraken account 1	18.10.2017 21:38:26	BTC	ETH	0.70259	12.5	VQO72T-BNEVZ-GN2YRH	0.001875	BTC	trade

Kraken	Kraken account 1	20.10.2017 01:48:45		BTC		0.2	CCCJIC5-TTWEOY-XMVPYA	0.001	BTC	withdrawal

This is the format by Bitfinex's trade history file:

\# Pair	Amount	Price	Fee	FeeCurrency	Date

In [5]:
###########################################
### I. Create empty panda frame with the right format
### We will append all entries (data frames) to this template frame
###########################################
df_table_Bitfinex = pd.DataFrame(columns = columns)
# index variable we need when appending elements
ix = 0

###########################################
### II. Handle deposits
###########################################
for i1 in range(len(df_deposits)):
    
    ###########################################
    # get deposit (trade) date and convert it to German notation
    ###########################################
    str_trade_date = df_deposits.iloc[i1, 7]
    str_trade_date_German = convert_trade_date(str_trade_date)
    
    ###########################################
    # collect all deposit (trade) data in a vector
    ###########################################
    vec = [['Bitfinex',               # exchange_name
            str_account_name_Bitfinex,# account_name
            str_trade_date_German,    # trade_date
            df_deposits.iloc[i1, 1],  # buy_asset
            '',                       # sell_asset
            df_deposits.iloc[i1, 3],  # buy_amount
            '',                       # sell_amount
            df_deposits.iloc[i1, 0],  # exchange_order_id - THIS COULD ALSO BE THE TXID df.iloc[i1, 6] ??
            0,                        # fee
            df_deposits.iloc[i1, 1],  # fee_asset
            'deposit']]               # transaction_type

    ###########################################
    # convert vector into pandas data frame and append it to table
    ###########################################
    df_i = pd.DataFrame(vec, columns = columns, index = [ix])
    df_table_Bitfinex = df_table_Bitfinex.append(df_i)
    ix += 1

###########################################
### III. Handle withdrawals
###########################################
for i2 in range(len(df_withdrawals)):

    str_trade_date = df_withdrawals.iloc[i2, 7]
    str_trade_date_German = convert_trade_date(str_trade_date)

    no_sell_amount = df_withdrawals.iloc[i2, 3]
    ###########################################
    ### withdrawals are declared negative in the withdrawal reports by Bitfinex
    ###########################################
    if no_sell_amount < 0:
        no_sell_amount = - no_sell_amount
    
    ###########################################
    ## UNFORTUNATELY, BITFINEX DOESN'T STATE IN THEIR REPORTS THE WITHDRAWAL FEE
    ###########################################
    
    vec = [['Bitfinex',                  # exchange_name
            str_account_name_Bitfinex,   # account_name
            str_trade_date_German,       # trade_date
            '',                          # buy_asset
            df_withdrawals.iloc[i2, 1],  # sell_asset
            '',                          # buy_amount
            no_sell_amount,              # sell_amount
            df_withdrawals.iloc[i2, 0],  # exchange_order_id - THIS COULD ALSO BE THE TXID df.iloc[i1, 6] ??
            0,                           # fee - UNFORTUNATELY, BITFINEX DOESN'T STATE IN THEIR REPORTS THE WITHDRAWAL FEE
            df_withdrawals.iloc[i2, 1],  # fee_asset
            'withdrawal']]               # transaction_type

    df_i = pd.DataFrame(vec, columns = columns, index = [ix])
    df_table_Bitfinex = df_table_Bitfinex.append(df_i)
    ix += 1

###########################################
### IV. Handle trades
###########################################
for i3 in range(len(df_trades)):

    str_trade_date = df_trades.iloc[i3, 6]
    str_trade_date_German = convert_trade_date(str_trade_date)
    
    ###########################################
    # get trade pair
    ###########################################
    str_pair = df_trades.iloc[i3, 1]
    idx_slash = str_pair.find('/')

    ###########################################
    # get the sign of the trade which signals if buy or sell trade
    ###########################################
    sign_trade = np.sign(df_trades.iloc[i3, 2])
    
    if sign_trade == 1:

        ###########################################
        ## Buy trades are noted as follows:
        ###########################################
        ## 1234567,XRP/BTC,200.0,0.00007111,-0.4,XRP,2017-09-25 19:04:40
        ## 200 units of XRP have been bought where one XRP cost 0.00007111 BTC.
        ## The fee of this trade was 0.4 XRP.
        ## So 200 * 0.00007111 = 0.014222 BTC have been sold.
        ###########################################
        
        str_buy_asset = str_pair[0 : idx_slash]
        str_sell_asset = str_pair[idx_slash + 1 : ]
        no_buy_amount = df_trades.iloc[i3, 2]
        no_sell_amount = df_trades.iloc[i3, 2] * df_trades.iloc[i3, 3]    

    elif sign_trade == -1:

        ###########################################
        ## Sell trades are noted with negative values
        ###########################################
        ## Example: 1234567,LTC/BTC,-2.0,0.018693,-0.00007477,BTC,2017-11-23 00:36:12
        ## 2 LTC have been sold against BTC where one LTC was sold for 0.018693 BTC.
        ## So 2 * 0.018693 = 0.037386 BTC have been bought.
        ###########################################
        
        str_buy_asset = str_pair[idx_slash + 1 : ]
        str_sell_asset = str_pair[0 : idx_slash]
        no_buy_amount = - df_trades.iloc[i3, 2] * df_trades.iloc[i3, 3]
        no_sell_amount = - df_trades.iloc[i3, 2]
        
    ###########################################
    ## get trade fee and make sign positive since 
    ## Cryptotax.io expects fees to be positive
    ###########################################
    no_fee = df_trades.iloc[i3, 4]
    if no_fee < 0:
        no_fee = - no_fee
    
    vec = [['Bitfinex',                  # exchange_name
            str_account_name_Bitfinex,   # account_name
            str_trade_date_German,       # trade_date
            str_buy_asset,               # buy_asset
            str_sell_asset,              # sell_asset
            no_buy_amount,               # buy_amount
            no_sell_amount,              # sell_amount
            df_trades.iloc[i3, 0],       # exchange_order_id - THIS COULD ALSO BE THE TXID df.iloc[i1, 6] ??
            no_fee,                      # fee
            df_trades.iloc[i3, 5],       # fee_asset
            'trade']]                    # transaction_type

    df_i = pd.DataFrame(vec, columns = columns, index = [ix])
    df_table_Bitfinex = df_table_Bitfinex.append(df_i)
    ix += 1

###########################################
## show table
###########################################
print(df_table_Bitfinex.iloc[:, 3 :])

Empty DataFrame
Columns: [buy_asset, sell_asset, buy_amount, sell_amount, exchange_order_id, fee, fee_asset, transaction_type]
Index: []


### Export table into excel file that can be imported to CryptoTax.io

#### For some reason the following block doesn't export the excel file right away so we simply run this block twice.

In [6]:
for i1 in range(2):
    # print(i1)
    writer = pd.ExcelWriter('Bitfinex/Bitfinex_2017.xlsx')
    df_table_Bitfinex.to_excel(writer, 'Bitfinex_2017', index = False, header = True)

### Parsing and conversion of Coinbase files

#### The format by Coinbase is different than the format by Bitfinex:

0  'Timestamp',

1  'Balance',

2  'Amount',

3  'Currency',

4  'To',

5  'Notes',

6  'Instantly Exchanged', 

7  'Transfer Total',

8  'Transfer Total Currency',

9  'Transfer Fee',

10 'Transfer Fee Currency',

11 'Transfer Payment Method', 

12 'Transfer ID',

13 'Order Price',

14 'Order Currency', 

15 'Order BTC', 

16 'Order Tracking Code', 

17 'Order Custom Parameter', 

18 'Order Paid Out', 

19 'Recurring Payment ID', 

20 'Coinbase ID (visit https://www.coinbase.com/transactions/[ID] in your browser)', 

21 'Bitcoin Hash (visit https://www.coinbase.com/tx/[HASH] in your browser for more info)\n']

Example:

['2017-09-24 01:03:37 -0700', 

'0.5', 

'0.5', 

'ETH', 

'RandomETHAddressabcde12345', 

'Bought 0.5000 ETH for â‚¬172.83 EUR.', 

'false', 

'172.83', 

'EUR', 

'14.28', 

'EUR', 

'EUR Wallet', 

'RandomTransferID98765', 

'""', 

'""', 

'""', 

'""', 

'""', 

'""', 

'""', 

'RandomCoinbaseID76543', 

'""\n']

In [7]:
df_table_Coinbase = pd.DataFrame(columns = columns)
ix = 0

###########################################
### Gg through all files generated by Coinbase 
### and listed in variable str_file_names_Coinbase
###########################################
for str_file_name in str_file_names_Coinbase:

    bool_file_access = os.access(str_file_name, os.R_OK)
    if bool_file_access == True:
        print('\nNow reading file ' + str_file_name + ' generated by Coinbase.\n')
    else:
        print('\nWARNING: File ' + str_file_name + ' generated by Coinbase does not exist or cannot be read!')
        continue
    
    ###########################################
    ### get name of wallet | asset
    ###########################################
    idx_file_name_hyphen = str_file_name.find('-')
    idx_file_name_dot = str_file_name.find('.')
    str_name_wallet = str_file_name[idx_file_name_hyphen + 1 : idx_file_name_dot]
    
    ###########################################
    ### open file and walk through all lines
    ###########################################
    with open(str_file_name) as f:
        
        for i, line in enumerate(f):
            
            ###########################################
            ### obtain indicators such as hyphens and certain phrases
            ###########################################
            line = line.replace('â‚¬', '')
            line_split = line.split(',')
            element_01 = line_split[0]
            found_hyphen = element_01.find('-')
            found_paid = element_01.find('Your digital currency')
            found_congrats = line.find('Congrats!')
            found_find_out = line.find('out more on')
            
            ###########################################
            ### we look for relevant lines that contain information about deposits, withdrawals, and trades
            ###########################################
            if found_hyphen > 0 or found_paid > 0 or found_find_out > 0:

                # print(len(line_split))
                # print(line_split)

                ###########################################
                ### a line "Paid for with EUR Wallet. Your digital currency will arrive by the end of day on Friday Dec 23, 2017. ..."
                ### is preceded by an empty line and "2017-11-23 06:18:47 -0800,0.15,0.15,BTC,RandomBTCAddressabcde12345,"Bought 0.15 BTC for â‚¬XXX.00 EUR."
                ### The information of this buy activity is now complete.
                ###########################################
                if found_paid > 0:
                    
                        line_split = line_split_store + line_split[2 :]
                        found_hyphen = 1
                        entry_complete = True

                ###########################################
                ### a line "Find out more on the https://www.coinbase.com/invite_friends page.",false,"","","","","","","","","","","","","",RandomCoinbaseID12345,"""
                ### is preceded by an empty line and 
                ### "2017-09-24 01:08:44 -0700,0.002,0.002,BTC,your_account_name@gmail.com,"Congrats! You just earned a $10 (â‚¬8) bonus for accepting your friend Mr Coin Base's Coinbase invite! They also got $10 (â‚¬8) of bitcoin for telling you about Coinbase."
                ### The information of this deposit activity is now complete.
                ###########################################
                if found_find_out > 0:
                    
                        line_split = line_split_store + line_split[1 :]
                        found_hyphen = 1
                        entry_complete = True
                       
                ###########################################
                ### found a line that starts with a date such as "2017-11-04 12:02:14 -0800"
                ### which signals a regular deposit, withdrawal, or trade activity which per se is complete.
                ###########################################
                if found_hyphen > 0:
                    
                    str_trade_date = line_split[0]
                    str_trade_date_German = convert_trade_date(str_trade_date)
                    idx_space = [pos for pos, char in enumerate(str_trade_date_German) if char == ' ']
                    str_trade_date_German = str_trade_date_German[: idx_space[1]]
                    # print(str_trade_date_German)

                ###########################################
                ### for now we assume the activity is a buy action.
                ### if it turns out to be a withdrawal or sell action,
                ### we will set the correct values below.
                ###########################################
                no_buy_amount = line_split[2]

                ###########################################
                ### go on if a line with complete info on the activity is found or information is complete.
                ###########################################
                if found_hyphen > 0 or entry_complete == True:
                    
                    ###########################################
                    ## Four digit long numbers are designated with a ','
                    ## which leads to different splitting that is the split up line can contain either 22 or 23 elements.
                    ###########################################
                    ## Hence, we need to pick the right entries in the split up vector
                    ###########################################
                    if len(line_split) == 23:
                        # print(len(line_split))
                        # print(line_split)
                        
                        ###########################################
                        ## we either found a deposit or buy action
                        ###########################################
                        if float(no_buy_amount) > 0:
                            
                            ###########################################
                            ## no_buy_amount = line_split[2] does not change.
                            ###########################################
                            
                            str_buy_asset = line_split[3]
                            str_sell_asset = line_split[9]
                            no_sell_amount = line_split[8]
                                                        
                            if str_sell_asset == '""' or str_sell_asset == '""""':
                                trade_type = 'deposit'
                            else:
                                trade_type = 'buy'

                        ###########################################
                        ## we either found a withdrawal or sell action
                        ###########################################
                        else:
                            
                            ###########################################
                            ## no_buy_amount does change.
                            ###########################################
                            
                            str_buy_asset = line_split[9]
                            str_sell_asset = line_split[3]
                            no_sell_amount = - float(line_split[2])
                            no_buy_amount = line_split[8]
                            
                            if str_buy_asset == '""' or str_buy_asset == '""""':
                                trade_type = 'withdrawal'
                            else:
                                trade_type = 'sell'

                        ###########################################
                        ## get the fee, fee asset, order ID and declare information complete
                        ###########################################
                        no_fee = line_split[10]
                        order_id = line_split[21]
                        fee_asset = line_split[11]
                        line_split_store = []
                        entry_complete = True
                        
                    elif len(line_split) == 22:
                        # print(len(line_split))
                        # print(line_split)
                        
                        # str_sell_asset = line_split[8]
                        # no_sell_amount = line_split[7]
                        
                        ###########################################
                        ## we either found a deposit or buy action
                        ###########################################
                        if float(no_buy_amount) > 0:
                            
                            ###########################################
                            ## no_buy_amount = line_split[2] does not change.
                            ###########################################
                            
                            str_buy_asset = line_split[3]
                            str_sell_asset = line_split[8]
                            no_sell_amount = line_split[7]
                            
                            if str_sell_asset == '""' or str_sell_asset == '""""':
                                trade_type = 'deposit'
                            else:
                                trade_type = 'buy'
                                
                        ###########################################
                        ## we either found a withdrawal or sell action
                        ###########################################
                        else:
                            str_buy_asset = line_split[8]
                            str_sell_asset = line_split[3]
                            no_sell_amount = - float(line_split[2])
                            no_buy_amount = line_split[7]
                            
                            if str_buy_asset == '""' or str_buy_asset == '""""':
                                trade_type = 'withdrawal'
                            else:
                                trade_type = 'sell'
                        
                        ###########################################
                        ## get the fee, fee asset, order ID and declare information complete
                        ###########################################
                        no_fee = line_split[9]
                        order_id = line_split[20]
                        fee_asset = line_split[10]
                        line_split_store = []
                        entry_complete = True
                        
                    ###########################################
                    ## the number of elements in this line is 6 or 7 so information of this activity is incomplete, yet.
                    ## we store the information in this line and will append it to the information coming up in two lines.
                    ## the next line is skipped because it is empty.
                    ###########################################
                    else:
                        # print(len(line_split))
                        # print(line_split)
                        line_split_store = line_split
                        entry_complete = False

                    if entry_complete == True:
                        
                        # print(no_buy_amount)
                        # print(trade_type)
                        ###########################################
                        ## Buy trade
                        ###########################################
                        if trade_type == 'buy' or trade_type == 'sell':
                            
                            vec = [['Coinbase',                  # exchange_name
                                    str_account_name_Coinbase,   # account_name
                                    str_trade_date_German,       # trade_date
                                    str_buy_asset,               # buy_asset
                                    str_sell_asset,              # sell_asset
                                    no_buy_amount,               # buy_amount
                                    no_sell_amount,              # sell_amount
                                    order_id,                    # exchange_order_id - THIS COULD ALSO BE THE TXID df.iloc[i1, 6] ??
                                    no_fee,                      # fee - UNFORTUNATELY, BITFINEX DOESN'T STATE IN THEIR REPORTS THE WITHDRAWAL FEE
                                    fee_asset,                   # fee_asset
                                    'trade']]                    # transaction_type

                        ###########################################
                        ## Withdrawal
                        ###########################################
                        elif trade_type == 'deposit':

                            # print(line_split)
                            # print(no_sell_amount)

                            ###########################################
                            ## UNFORTUNATELY, Coinbase DOESN'T STATE IN THEIR REPORTS THE WITHDRAWAL FEE
                            ###########################################

                            vec = [['Coinbase',                  # exchange_name
                                    str_account_name_Coinbase,   # account_name
                                    str_trade_date_German,       # trade_date
                                    str_buy_asset,               # buy_asset
                                    '',                          # sell_asset
                                    no_buy_amount,               # buy_amount
                                    '',                          # sell_amount
                                    order_id,                    # exchange_order_id - THIS COULD ALSO BE THE TXID df.iloc[i1, 6] ??
                                    0,                           # fee - UNFORTUNATELY, Coinbase DOESN'T STATE IN THEIR REPORTS THE WITHDRAWAL FEE
                                    str_buy_asset,               # fee_asset
                                    'deposit']]                  # transaction_type
                            
                        ###########################################
                        ## Withdrawal
                        ###########################################
                        elif trade_type == 'withdrawal':

                            # print(line_split)
                            # print(no_sell_amount)

                            ###########################################
                            ## UNFORTUNATELY, Coinbase DOESN'T STATE IN THEIR REPORTS THE WITHDRAWAL FEE
                            ###########################################

                            vec = [['Coinbase',                  # exchange_name
                                    str_account_name_Coinbase,   # account_name
                                    str_trade_date_German,       # trade_date
                                    '',                          # buy_asset
                                    str_sell_asset,              # sell_asset
                                    '',                          # buy_amount
                                    no_sell_amount,              # sell_amount
                                    order_id,                    # exchange_order_id - THIS COULD ALSO BE THE TXID df.iloc[i1, 6] ??
                                    0,                           # fee - UNFORTUNATELY, Coinbase DOESN'T STATE IN THEIR REPORTS THE WITHDRAWAL FEE
                                    str_sell_asset,              # fee_asset
                                    'withdrawal']]               # transaction_type

                        ###########################################
                        # convert vector into pandas data frame and append it to table
                        ###########################################
                        df_i = pd.DataFrame(vec, columns = columns, index = [ix])
                        df_table_Coinbase = df_table_Coinbase.append(df_i)
                        ix += 1
            
###########################################
## show table
###########################################
df_table_Coinbase.iloc[:, 2 :]

,trade_date,buy_asset,sell_asset,buy_amount,sell_amount,exchange_order_id,fee,fee_asset,transaction_type


### Export table into excel file that can be imported to CryptoTax.io

#### For some reason the following block doesn't export the excel file right away so we simply run this block twice.

In [8]:
str_file_name_excel = 'Coinbase/Coinbase_2017.xlsx'
for i1 in range(2):
    writer = pd.ExcelWriter(str_file_name_excel)
    df_table_Coinbase.to_excel(writer, 'Coinbase_2017', index = False, header = True)